# Forest Fire Burn Area Feature Selection
Written by: Marshall Mykietyshyn

Data hosted on [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Forest+Fires).

The purpose of this notebook is to perform feature selection on the forest fire burn area dataset, testing its affect on the results. This is a popular regression dataset found on both Kaggle and the UCI Machine Learning Repository. The two data sets are identical, but the one used here was downloaded from UCI. The target variable is burn area, which can be predicted by a set of feature variables listed below.
